In [1]:
import load_data_multitox as ld
import dataloaders_sigma as dl
from Model_train_test_regression import Net, EarlyStopping, train_regression, test_regression, train_classification, test_classification

import pandas as pd
import numpy as np

import torch
from torch.utils import data as td
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter

import sys 
import os
import glob

from sklearn.model_selection import train_test_split

from tensorboardX import SummaryWriter

import time
from sklearn.preprocessing import MinMaxScaler#StandardScaler

import json


# number of conformers created for every molecule
NUM_CONFS = 100

# amount of chemical elements taking into account
AMOUNT_OF_ELEM = 9

# amount of target values
TARGET_NUM = 29

#dataset folder
# DATASET_PATH="~/Tox21-MultiTox/MultiTox"
DATASET_PATH="./"

#logs path
LOG_PATH=os.path.join(DATASET_PATH,"logs_sigma_right")


#models path
MODEL_PATH=os.path.join(DATASET_PATH,"models_sigma_right")

In [2]:
EXPERIMENT_NUM=24

In [3]:
dir_path = os.path.join(LOG_PATH,'exp_'+str(EXPERIMENT_NUM))
os.makedirs(dir_path, exist_ok=True)
LOG_PATH = dir_path
dir_path = os.path.join(MODEL_PATH,'exp_'+str(EXPERIMENT_NUM))
os.makedirs(dir_path, exist_ok=True)
MODEL_PATH = dir_path

In [4]:
path="./"

In [5]:
with open(os.path.join(path,"logs_sigma_right",'exp_'+str(24),str(24)+'_parameters.json'),'r') as f:
  args = json.load(f)

In [6]:
# args['NUM_EXP']=str(EXPERIMENT_NUM)
# args['BATCH_SIZE']=64

# args['TRANSF']='w'
# args['SIGMA_TRAIN']=False

In [7]:
args

{'EPOCHS_NUM': 100,
 'PATIENCE': 25,
 'SIGMA': 1.4,
 'BATCH_SIZE': 128,
 'TRANSF': 'g',
 'NUM_EXP': '24',
 'VOXEL_DIM': 50,
 'LEARN_RATE': 1e-05,
 'SIGMA_TRAIN': False}

# Tox21 check

In [8]:
conf_calc = ld.reading_sql_database(database_dir='/gpfs/gpfs0/a.alenicheva/Tox21/elements_9')

In [6]:
tox21_data = pd.read_csv('../Tox21_Neural_Net/database/tox21_10k_data_all_no_salts.csv')

In [7]:
tox21_data.shape

(7690, 13)

In [9]:
elems = set([])
for smiles in list(conf_calc.keys()):
    if 0 in conf_calc[smiles].keys():
        if 'coordinates' in conf_calc[smiles][0].keys():
            elems = elems | set(conf_calc[smiles][0]['coordinates'].keys())
        else:
            print(smiles)
    if len(conf_calc[smiles].keys())!=100:
        print(smiles, len(conf_calc[smiles].keys()))

CCCCCCCC[N+](CCCCCCCC)(CCCCCCCC)CCCCCCCC 74
[C-]#N
II
NO
[NH3+]O
C=O
[Br-] 99
CO


In [10]:
elems

{'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S'}

In [11]:
len(conf_calc)

7489

In [12]:
list(conf_calc.keys())[:10]

['C[n+]1c2cc(N)ccc2cc2ccc(N)cc21',
 'O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(Br)c([O-])c(Br)cc12',
 'CN(C)c1ccc(C(=C2C=CC(=[N+](C)C)C=C2)c2ccccc2)cc1',
 'CCN(CC)C(=S)SSC(=S)N(CC)CC',
 'CCCCCCCCNC(C)C(O)c1ccc(SC(C)C)cc1',
 'CCCCCCOc1ccc(C(=N)N(CCCC)CCCC)c2ccccc12',
 'COCC(=O)O[C@]1(CCN(C)CCCc2nc3ccccc3[nH]2)CCc2cc(F)ccc2[C@@H]1C(C)C',
 'O[C@@H](CNC[C@H](O)[C@H]1CCc2cc(F)ccc2O1)[C@@H]1CCc2cc(F)ccc2O1',
 'CC(C)(C)CC(C)(C)c1ccc(O)c(Cc2ccc(Cl)cc2Cl)c1',
 'CCCCCNC(=N)N/N=C/c1c[nH]c2ccc(OC)cc12']

In [16]:
conf_calc['C[n+]1c2cc(N)ccc2cc2ccc(N)cc21'][0]['coordinates']

{'C': [(-3.6268269856638984, -1.0375925343625578, -0.02359165959177123),
  (-2.4138089693296587, -1.7195390060281144, 0.04802283408855934),
  (-1.2020822626167345, -1.0171005271923719, 0.052807899127492175),
  (1.0426778304171557e-05, -1.7106172939141486, 0.09398429695263755),
  (1.2020949618846781, -1.0170331317601495, 0.05281363361112956),
  (2.4138291464117296, -1.7194057297669318, 0.04803058957657702),
  (3.6268392793793782, -1.0373921845712302, -0.023588517237407958),
  (3.652891960520005, 0.3471007663144677, -0.15088531054442594),
  (2.444748082315201, 1.0518595703547133, -0.08506159968536847),
  (1.1986740573974775, 0.39779530031740445, 0.02365028686073814)],
 'N': [(-4.867185016227522, 1.0309796333721681, -0.11477480817525337),
  (4.867174537683415, 1.0312452334279247, -0.11478520594885505)]}

In [27]:
conf_calc['O=C(CCl)c1ccc(Cl)cc1Cl'][0]['coordinates'].keys()

dict_keys(['C', 'Cl'])

In [14]:
conf_calc1 = ld.reading_sql_database(database_dir='/gpfs/gpfs0/a.alenicheva/Tox21/elements_6')

In [52]:
len(conf_calc1)

7347

In [17]:
conf_calc1['C[n+]1c2cc(N)ccc2cc2ccc(N)cc21'][0]['coordinates']

{'C': [(-3.624704689460949, -1.0400542624639746, -0.05604773612610772),
  (-2.4111786710972973, -1.7206793154248343, 0.018851783553119755),
  (-1.201936550446931, -1.0158418915971248, 0.07019916449832594),
  (1.1994908132435334e-05, -1.70955351242743, 0.10216782102088526),
  (1.2019507039585562, -1.0157521601115314, 0.07019740692399132),
  (2.4112035582265365, -1.7204989008955929, 0.01884567237841947),
  (3.624719640954574, -1.0397833209903073, -0.056055854932665045),
  (3.6586629246579894, 0.35032192721200195, -0.043732996383501116),
  (2.4471786865523066, 1.0525528689683572, -0.036762480784361035),
  (1.199329685225353, 0.3993838118365083, 0.05882516027655085)],
 'N': [(-4.858772969348557, 1.0223176233507927, -0.27026301226630467),
  (4.8587579313542255, 1.022679579078178, -0.27025868587662294)],
 'H': [(-0.8663963566244336, 2.8562394176823513, 0.8806435116357435),
  (0.8663442349563052, 2.856285904619843, 0.8806726146794072),
  (-2.517225571996689, 2.130519891410097, -0.141259755873

In [64]:
elems1 = set([])
for smiles in list(conf_calc1.keys()):
    if 'coordinates' in conf_calc1[smiles][0].keys():
        elems1 = elems1 | set(conf_calc1[smiles][0]['coordinates'].keys())
    if len(conf_calc1[smiles].keys())!=100:
        print(smiles)

In [65]:
elems1

{'C', 'Cl', 'H', 'N', 'O', 'S'}

In [53]:
elems - elems1

{'8', '9'}

In [60]:
set(conf_calc1.keys()) - set(conf_calc.keys())

{'CCCCC1(COC(=O)CCC(=O)O)C(=O)N(c2ccccc2)N(c2ccccc2)C1=O'}

In [59]:
set(conf_calc.keys()) - set(conf_calc1.keys())

{'C/C(=C(\\CCOC(=O)c1ccccc1)SS/C(CCOC(=O)c1ccccc1)=C(/C)N(C=O)Cc1cnc(C)nc1N)N(C=O)Cc1cnc(C)nc1N',
 'C/C(=C\\C(=O)OCCCCCCCCC(=O)O)C[C@@H]1OC[C@H](C[C@@H]2O[C@H]2[C@@H](C)[C@H](C)O)[C@@H](O)[C@H]1O',
 'C/C=C(/CC[C@@H](C)[C@H]1CC[C@H]2C3=CC[C@H]4[C@H](C)[C@@H](O)CC[C@]4(C)[C@H]3CC[C@@]21C)C(C)C',
 'C=CCNC1=C2C[C@@H](C)C[C@H](OC)[C@H](O)[C@@H](C)/C=C(\\C)[C@H](OC(N)=O)[C@@H](OC)/C=C\\C=C(/C)C(=O)NC(=CC1=O)C2=O',
 'C=CC[N+]1([C@H]2C[C@H]3[C@@H]4CC[C@H]5C[C@H](OC(C)=O)[C@@H](N6CCCCC6)C[C@]5(C)[C@H]4CC[C@]3(C)[C@H]2OC(=O)CC)CCCCC1',
 'C=CC[N@@+]12CC[C@@]34c5ccccc5N5/C=C6/[C@H]7C[C@H]8[C@@]9(CC[N@@+]8(CC=C)C/C7=C/CO)c7ccccc7N(/C=C(/[C@@H](C[C@@H]31)/C(=C\\CO)C2)[C@H]54)[C@@H]69',
 'C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H](N)C(C)C)[C@]2(C)[C@H](C)CC[C@]3(CCC(=O)[C@H]32)[C@@H](C)[C@@H]1O',
 'CC(=O)N(CC(O)CN(C(C)=O)c1c(I)c(C(=O)NCC(O)CO)c(I)c(C(=O)NCC(O)CO)c1I)c1c(I)c(C(=O)NCC(O)CO)c(I)c(C(=O)NCC(O)CO)c1I',
 'CC(=O)NC1(c2cccc(F)c2)CCN(CC[C@@]2(c3ccc(Cl)c(Cl)c3)CN(c3ccccc3)C(=O)CO2)CC1',
 'C

# End check

In [7]:
f_log=open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'w')
f_log.close()
start_time=time.time()
writer=SummaryWriter(LOG_PATH)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
    f_log.write('Using device:'+str(device)+'\n')
print()
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

    with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
        f_log.write(torch.cuda.get_device_name(0)+'\n'+'Memory Usage:'+'\n'+'Allocated:'+str(round(torch.cuda.memory_allocated(0)/1024**3,1))+ 'GB'+'\n'+'Cached:   '+str(round(torch.cuda.memory_cached(0)/1024**3,1))+'GB'+'\n')
print('Start loading dataset...')
with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
    f_log.write('Start loading dataset...'+'\n')
# get dataset without duplicates from csv
data = pd.read_csv(os.path.join(DATASET_PATH,'database', 'data','MultiTox.csv'))
props = list(data)[1:]
scaler = MinMaxScaler()
data[props]=scaler.fit_transform(data[props])

# create elements dictionary
#     elements = ld.create_element_dict(data, amount=AMOUNT_OF_ELEM+1)
elements={'N':0,'C':1,'Cl':2,'I':3,'Br':4,'F':5,'O':6,'P':7,'S':8}

# read databases to dictionary
#     conf_calc = ld.reading_sql_database(database_dir='./dat/')
with open(os.path.join('./','many_elems.json'), 'r') as fp:
    conf_calc = json.load(fp)



Using device: cuda:0

GeForce GTX 1080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
Start loading dataset...


In [8]:
keys=list(conf_calc.keys())
print ('Initial dataset size = ', len(keys))
with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
    f_log.write('Initial dataset size = '+str(len(keys))+'\n')
new_conf_calc={}
for smiles in conf_calc.keys():
    for conf_num in conf_calc[smiles]:
        if smiles in new_conf_calc.keys():
            new_conf_calc[smiles][int(conf_num)]=conf_calc[smiles][conf_num]
        else:
            new_conf_calc[smiles]={}
            new_conf_calc[smiles][int(conf_num)]=conf_calc[smiles][conf_num]

conf_calc=new_conf_calc

elems = []
for key in keys:
    conformers=list(conf_calc[key].keys())
    for conformer in conformers:
        try:
            energy = conf_calc[key][conformer]['energy']
            elems = list(set(elems+list(conf_calc[key][conformer]['coordinates'].keys())))
        except:
            del conf_calc[key][conformer]
    if set(conf_calc[key].keys())!=set(range(100)):
          del conf_calc[key]
    elif conf_calc[key]=={}:
        del conf_calc[key]

print ('Post-processed dataset size = ', len(list(conf_calc.keys())))
with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
    f_log.write('Post-processed dataset size = '+str(len(list(conf_calc.keys())))+'\n')


Initial dataset size =  13091
Post-processed dataset size =  13084
Dataset has been loaded,  184  s
Neural network initialization...


In [ ]:
# create indexing and label_dict for iteration
indexing, label_dict = ld.indexing_label_dict(data, conf_calc)
print('Dataset has been loaded, ', int(time.time()-start_time),' s')
with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
    f_log.write('Dataset has been loaded, '+str(int(time.time()-start_time))+' s'+'\n')

start_time=time.time()
# create train and validation sets' indexes
print('Neural network initialization...')
with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
    f_log.write('Neural network initialization...'+'\n')


In [9]:
train_indexes, test_indexes, _, _ = train_test_split(np.arange(0, len(conf_calc.keys())),
                                                     np.arange(0, len(conf_calc.keys())), test_size=0.2,
                                                     random_state=115)
train_set = dl.Cube_dataset(conf_calc, label_dict, elements, indexing, train_indexes, dim = args['VOXEL_DIM'])
train_generator = td.DataLoader(train_set, batch_size=args['BATCH_SIZE'], shuffle=True)

test_set = dl.Cube_dataset(conf_calc, label_dict, elements, indexing, test_indexes, dim = args['VOXEL_DIM'])
test_generator = td.DataLoader(test_set, batch_size=args['BATCH_SIZE'], shuffle=True)

model = Net(dim=args['VOXEL_DIM'], num_elems=AMOUNT_OF_ELEM, num_targets=TARGET_NUM, elements=elements, transformation=args['TRANSF'],device=device,sigma_0 = args['SIGMA'],sigma_trainable = args['SIGMA_TRAIN'])


In [10]:
from torch.autograd import Variable
threshold = 5
for i, (molecule, curr_target) in enumerate(train_generator):
        molecule = Variable(molecule.to(device),requires_grad=True)
        if i > threshold:
            break
# model = Net(dim=args_dict['VOXEL_DIM'], num_elems=AMOUNT_OF_ELEM, num_targets=TARGET_NUM, elements=elements, transformation=args_dict['TRANSF'],device=device,sigma_0 = args_dict['SIGMA'],sigma_trainable = args_dict['SIGMA_TRAIN'], x_trainable=True, x_input=torch.randn(1,9,50,50,50))
# model=model.to(device)
# model.load_state_dict(torch.load(os.path.join(MODEL_PATH_LOAD,'checkpoint.pt')))
model.x_input=Parameter(molecule,requires_grad=True)

In [2]:
def plot_visualization_input_as_parameter(model,elements,losses, epoch):
    import matplotlib.pyplot as plt
    inv_elems = {v: k for k, v in elements.items()}    
    data=model.x_input
    molecules = data.cpu().detach().sum(dim=0)
    fig = plt.figure(figsize=(10,15),constrained_layout=True)
    gs = fig.add_gridspec(4, 3)
    for i,grad in enumerate(molecules):
        f_ax = fig.add_subplot(gs[i//3,i%3])
        f_ax.imshow(grad.sum(dim=0))
        f_ax.set_title(inv_elems[i],fontsize=25)

    f_ax = fig.add_subplot(gs[-1, :])
    f_ax.plot(5*np.arange(0,len(losses),1),losses)
    f_ax.set_title('Loss function',fontsize=25)
    f_ax.set_xlabel('epochs',fontsize=25)
    f_ax.set_ylabel('loss',fontsize=25)
    fig.suptitle('Atom types in molecule',fontsize=25)
    
    plt.show()
#     fig.savefig(os.path.join(LOG_PATH_SAVE,'images','img_'+str(epoch))+'.png',dpi=150,format='png')
    _ = plt.clf()

In [3]:
plot_visualization_input_as_parameter(model,elements,[], 1)

NameError: name 'model' is not defined

In [8]:
MODEL_PATH

'./models_sigma_right/exp_24'

In [11]:
 model.load_state_dict(torch.load(os.path.join('./models_sigma_right/exp_43','checkpoint.pt')))

<All keys matched successfully>

In [ ]:
train_set = dl.Cube_dataset(conf_calc, label_dict, elements, indexing, train_indexes, dim = args['VOXEL_DIM'])
train_generator = td.DataLoader(train_set, batch_size=args['BATCH_SIZE'], shuffle=True)

test_set = dl.Cube_dataset(conf_calc, label_dict, elements, indexing, test_indexes, dim = args['VOXEL_DIM'])
test_generator = td.DataLoader(test_set, batch_size=args['BATCH_SIZE'], shuffle=True)

model = Net(dim=args['VOXEL_DIM'], num_elems=AMOUNT_OF_ELEM, num_targets=TARGET_NUM, elements=elements, transformation=args['TRANSF'],device=device,sigma_0 = args['SIGMA'],sigma_trainable = args['SIGMA_TRAIN'])


if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    print ('Run in parallel!')
    with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
        f_log.write('Run in parallel!'+'\n')

# Construct our model by instantiating the class defined above

model=model.to(device)

for (batch, target) in train_generator:
    batch = batch.to(device)
    target = target.to(device)
    with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
        f_log.write('Batch to device!'+'\n')
    print('Batch to device!')
    output = model(batch)
    with open(os.path.join(LOG_PATH,args['NUM_EXP']+'_logs.txt'),'a') as f_log:
        f_log.write('Batch output!'+'\n')
    print('Batch output!')
    break

In [1]:
import os

In [2]:
os.listdir('./models_sigma_right/exp_24')

['24_model_60',
 '24_model_30',
 '24_model_20',
 '24_model_10',
 'checkpoint.pt',
 '24_model_50',
 '24_model_40',
 'exp_24']